In [14]:
using JuMP, NamedArrays

## Stage 1

#### Wining probability function

In [34]:
function wp(x, y, idx)
    if x > y 
        return 1
    elseif x < y
        return -1
    else 
        return 0
    end  
end

wp (generic function with 1 method)

#### Case 1: no order, check who fights whom

In [29]:
size = 3
team1 = [1.5 2.5 3.5]
team2 = [1 2 3] # our team
;

In [30]:
m = Model()

@variable(m, x[1:size, 1:size], Bin)

@constraint(m, supply[k in 1:size], sum(x[k, j] for j=1:size) == 1)
@constraint(m, demand[k in 1:size], sum(x[i, k] for i=1:size) == 1)

@objective(m, Max, sum(x[i, j] * wp(team2[i], team1[j], 1) for i=1:size, j=1:size))
                                    
solve(m)
xopt = getvalue(x)
println("Strategy: ", )
for i in 1:size
    for j in 1:size
        if xopt[i, j] != 0
            println(team1[i], " - ", team2[j])
        end
    end
end
println("Score: ", getobjectivevalue(m))

Strategy: 
1.5 - 3
2.5 - 1
3.5 - 2
Score: 1.0


#### Case 2: order matters - fix team 1's plan

In [43]:
function wp_order(a, b, idx)
    # first move => +0.5
    x = a
    y = b
    if idx % 2 == 1
        x = a + 0.75
    else 
        y = b + 0.75
    end
    if x > y 
        return 1
    elseif x < y
        return -1
    else 
        return 0
    end  
end

wp_order (generic function with 1 method)

In [44]:
size = 3
team1 = [1.5 2.5 3.5]
team2 = [1 2 3] # our team
plan1 = [1 2 3] # fixed plan
;

In [47]:
m = Model()

@variable(m, x[1:size, 1:size], Bin)

@constraint(m, supply[k in 1:size], sum(x[k, j] for j=1:size) == 1)
@constraint(m, demand[k in 1:size], sum(x[i, k] for i=1:size) == 1)

@objective(m, Max, sum(x[i, j] * wp_order(team2[j], team1[plan1[i]], i + 1) for i=1:size, j=1:size))
                                    
solve(m)
xopt = getvalue(x)
println("Strategy: ", )
for i in 1:size
    for j in 1:size
        if xopt[i, j] != 0
            println(team1[i], " - ", team2[j])
        end
    end
end
println("Score: ", getobjectivevalue(m))

Strategy: 
1.5 - 3
2.5 - 2
3.5 - 1
Score: 1.0


#### Probabilistic model

In [118]:
#the positional advantage and the ratings difference
# higher elo, first move better
function win_prob(a, b, idx)
    first = 0.28
    if idx % 2 == 1
        x = a - b + 35
    else
        x = a - b - 35
    end
    return 1/(1 + 10^(-x/400)) 
end

win_prob (generic function with 1 method)

In [119]:
function score(self, opp, idx)
    prob = win_prob(self, opp, idx)
    return prob
end

score (generic function with 1 method)

In [120]:
# Russia vs USA
size = 4
team1 = [2808 2769 2754 2740]
team2 = [2808 2789 2782 2674] # our team
plan1 = [2 1 4 3] # fixed plan
;

In [121]:
plan2 = team2
size = 4
result = sum(score(team2[i], team1[plan1[i]], i + 1) for i=1:size)
println(result)

1.9744377523957244


In [122]:
m = Model()

@variable(m, x[1:size, 1:size], Bin)

@constraint(m, supply[k in 1:size], sum(x[k, j] for j=1:size) == 1)
@constraint(m, demand[k in 1:size], sum(x[i, k] for i=1:size) == 1)

@objective(m, Max, sum(x[i, j] * score(team2[j], team1[plan1[i]], i + 1) for i=1:size, j=1:size))
                                    
solve(m)
xopt = getvalue(x)
println("Strategy: ", )
for i in 1:size
    for j in 1:size
        if xopt[i, j] != 0
            println(team1[plan1[i]], " - ", team2[j])
        end
    end
end
println("Score: ", getobjectivevalue(m))

Strategy: 
2769 - 2674
2808 - 2789
2740 - 2808
2754 - 2782
Score: 1.9812246886886204


## Stage 2 - Nash equilibrium

Data - 2 fingers Morra

In [111]:
size = 2
# calculate matrix incidences (max of mine is min of theirs or zero-sum game)
myProb = [2 -3; -3 4]
oppProb = [-2 3; 3 -4]

2×2 Array{Int64,2}:
 -2   3
  3  -4

Model

In [113]:
# Our strategy P
m = Model()

@variable(m, 1>= p[1:size] >=0)
@variable(m, t)

@constraint(m, sum(p) == 1)
@constraint(m, min_con[k in 1:size], sum(p[i] * myProb[i, k] for i=1:size) >= t)
# Max min (g1, g2, ..., gn)
@objective(m, Max, t)

println(m)
solve(m)

popt = getvalue(p)
println("Strategy: ", popt)
println("E Score - Mine: ", getobjectivevalue(m))

Max t
Subject to
 p[1] + p[2] = 1
 2 p[1] - 3 p[2] - t ≥ 0
 -3 p[1] + 4 p[2] - t ≥ 0
 0 ≤ p[i] ≤ 1 ∀ i ∈ {1,2}
 t free

Strategy: [0.583333,0.416667]
E Score - Mine: -0.08333333333333359


In [114]:
# Oppponents's strategy Q
m = Model()

@variable(m, 1>= p[1:size] >=0)
@variable(m, t)

@constraint(m, sum(p) == 1)
@constraint(m, min_con[k in 1:size], sum(p[i] * oppProb[i, k] for i=1:size) >= t)
# Max min (g1, g2, ..., gn)
@objective(m, Max, t)

println(m)
solve(m)

popt = getvalue(p)
println("Strategy: ", popt)
println("O Score - theirs: ", getobjectivevalue(m))

Max t
Subject to
 p[1] + p[2] = 1
 -2 p[1] + 3 p[2] - t ≥ 0
 3 p[1] - 4 p[2] - t ≥ 0
 0 ≤ p[i] ≤ 1 ∀ i ∈ {1,2}
 t free

Strategy: [0.583333,0.416667]
O Score - theirs: 0.08333333333333359


### Chess match

In [115]:
function match_score(myPlan, oppPlan, white)
    length = size
    return sum(score(team2[myPlan[i]], team1[oppPlan[i]], i + 1) for i=1:length)
end

match_score (generic function with 1 method)

#### Generate matrix

In [116]:
# one is enough!! No of games - 1's score = 2's score

